In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import collections
import seaborn as sns

from matplotlib import pylab

import bnpy

FIG_SIZE = (3, 3)
pylab.rcParams['figure.figsize'] = FIG_SIZE

In [ ]:
from matplotlib import pylab

In [ ]:
m1 = np.array((1, 2))
cov1 = [[1, 0], [0, 1]]
component1 = np.random.multivariate_normal(m1, cov1, 100)

m2 = np.array((-2, -1))
cov2 = [[1, 0], [0, 1]]
component2 = np.random.multivariate_normal(m2, cov2, 100)

m3 = np.array((-2, 5))
cov3 = [[2, 1], [3, 4]]
component3 = np.random.multivariate_normal(m3, cov3, 100)

colors = ['#f8c313',
          '#bf0143',
          '#027f80']

plt.scatter(component1[:, 0], component1[:, 1], color=colors[0])
plt.scatter(component2[:, 0], component2[:, 1], color=colors[1])
plt.scatter(component3[:, 0], component3[:, 1], color=colors[2])

data = np.vstack((component1, component2, component3))

samp_colors = []
for i, d in enumerate([component1, component2, component3]):
    for _ in range(d.shape[0]):
        samp_colors.append(colors[i])
        
samp_colors = np.array(samp_colors)

In [ ]:
import matplotlib

In [ ]:
matplotlib.colors.to_rgba('#f8c313')

In [ ]:
np.where(samp_colors[:-1] != samp_colors[1:])

In [ ]:
len(data)

In [ ]:
sampColors = {(0, 100): colors[0],
              (100, 200): colors[1],
              (200, 300): colors[2]}

In [ ]:
samp_colors = []
for _ in range(0, 100):
    samp_colors.append(matplotlib.colors.to_rgba(colors[0]))
    
for _ in range(100, 200):
    samp_colors.append(matplotlib.colors.to_rgba(colors[1]))
    
for _ in range(200, 300):
    samp_colors.append(matplotlib.colors.to_rgba(colors[2]))

In [ ]:
sampColors

In [ ]:
dataset = bnpy.data.XData(data)

In [ ]:
merge_kwargs = dict(
    m_startLap=10,
    m_pair_ranking_procedure='total_size',
    m_pair_ranking_direction='descending',
    )

delete_kwargs = dict(
    d_startLap=10,
    d_nRefineSteps=50,
    )

In [ ]:
help(bnpy.viz.PlotComps.plotCompsFromHModel)

In [ ]:
def show_clusters_over_time(
        task_output_path=None,
        query_laps=[0, 1, 2, 10, 20, None],
        nrows=2,
        samp_colors=(0.3, 0.3, 0.3)):
    '''
    '''
    colors=['gray', 'gray', 'gray', 'gray', 'gray']
    ncols = int(np.ceil(len(query_laps) // float(nrows)))
    fig_handle, ax_handle_list = pylab.subplots(
        figsize=(FIG_SIZE[0] * ncols, FIG_SIZE[1] * nrows),
        nrows=nrows, ncols=ncols, sharex=True, sharey=True)
    for plot_id, lap_val in enumerate(query_laps):
        cur_model, lap_val = bnpy.load_model_at_lap(task_output_path, lap_val)
        cur_ax_handle = ax_handle_list.flatten()[plot_id]
        bnpy.viz.PlotComps.plotCompsFromHModel(
            cur_model, 
            dataset=dataset, 
            ax_handle=cur_ax_handle,
            Colors=colors,
            samp_colors=samp_colors)
        cur_ax_handle.set_xlim([-4.5, 4.5])
        cur_ax_handle.set_xlabel("lap: %d" % lap_val)
    pylab.tight_layout()
    plt.savefig("img/cluster-ocver-time.svg", format='svg', bbox_inches='tight')

In [ ]:
b_trained_model, b_info_dict = bnpy.run(
    dataset, 'DPMixtureModel', 'Gauss', 'memoVB',
    output_path=('/tmp/StandardNormalK1/' +
        'trymoves-K=%d-gamma=%s-ECovMat=%s*eye-moves=birth,merge,shuffle/' % (
            5, 5.0, 1.0)),
    nLap=100, nTask=1, nBatch=1,
    gamma0=5.0, sF=1.0, ECovMat='eye',
    K=5, initname='randexamplesbydist',
    moves='birth,merge,shuffle')

In [ ]:
show_clusters_over_time(b_info_dict['task_output_path'], samp_colors=sampColors)